In [ ]:
import sys
import os
from pathlib import Path

# Caminho absoluto para a raiz do projeto (um nível acima de notebooks)
project_root = os.path.abspath("..")
if project_root not in sys.path:
    sys.path.append(project_root)


In [ ]:
from config.utils import *
